In [45]:
import uuid
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lxml import etree
from skimage import draw
from PIL import Image, ImageDraw, ImageFont

from svgpathtools import svg2paths, svg2paths2
from IPython.core.display import HTML, SVG, display

In [8]:
# !pip install jedi==0.17.2

#### absolute gradients

In [11]:
OUTLINE_WEIGHT = 2
GRADIENT_WEIGHT = 1

In [12]:
fp = '../data/test2.svg'
paths, attributes, svg_attributes = svg2paths2(fp)

In [13]:
MIN_X, MIN_Y, W, H = [int(float(x)) for x in svg_attributes['viewBox'].split(' ')]
MIN_X, MIN_Y, W, H

(-271, 490, 834, 562)

In [146]:
from svgpathtools import Line, CubicBezier, parse_path

In [15]:
def normalize_path(d):
    qs = [x.strip() for x in re.split(r'(M|Q|L|Z)', d, )[1:]]
    
    tups = []
    identifier = None
    for q in qs:
        if q:
            if len(q)==1:
                identifier = q
            else:
                ls = [float(x) for x in q.split(' ')]

                if identifier in ['Q']:
                    tups.append((identifier, (ls[0] - MIN_X, ls[1] - MIN_Y, ls[2] - MIN_X, ls[3] - MIN_Y)))
                elif identifier in ['L', 'M']:
                    tups.append((identifier, (ls[0] - MIN_X, ls[1] - MIN_Y)))
                else:
                    raise RuntimeError(f'unseen q: {q}')
                identifier = None

    d_str = ''
    for (identifier, ls) in tups:
        d_str += f'{identifier} '
        for l in ls: d_str += f'{round(l, 3)} '

    return d_str.strip()

In [16]:
# normalize_path(attributes[0]['d'])

In [17]:
174.947 + MIN_Y

664.947

In [18]:
def get_path_xy(p):
    s = re.sub(r'\(|\)|j', r'', str(p.start))
    x, y = s.split('+')
    x, y = float(x) - MIN_X, float(y) - MIN_Y
    return y, x # return in row, col format

def get_curve(paths):
    return [get_path_xy(p) for p in paths]

def get_curves(svg_paths):
    return [get_curve(ps) for ps in svg_paths]

In [303]:
def get_intersection_ts(curve, boundary):
    intersects = curve.intersect(boundary)
    if not intersects: return None, None
    if len(intersects) == 1:
        i = intersects[0]
        t = i[1][2]
        if t < .5:
            return t, None
        else:
            return None, t
    return min(intersects[0][1][2], intersects[1][1][2]), max(intersects[0][1][2], intersects[1][1][2])

def get_outlines(attbs, curves):
    outlines = []
    for i, attb in enumerate(attbs):
        o = {}
        if 'stroke-width' in attb and int(float(attb['stroke-width'])) == OUTLINE_WEIGHT:
            o = {'index': i, 'bounds':curves[i], 'metadata': attb}
#             print(curves[i]['bounds'])
            o['svg_border'] = parse_path(normalize_path(attb['d']))
            o['polygon'] = draw.polygon([r for r, c in o['bounds']],
                                        [c for r, c in o['bounds']])
            o['mask'] = np.full((H, W), False)
            o['mask'][o['polygon'][0], o['polygon'][1]] = True
            o['gradients'], o['paths'], o['colors'], o['svg_paths'] = [], [], [], []
        if o:
            outlines.append(o)
    return outlines

def add_gradients(attbs, curves, outlines):
    idxs = {o['index'] for o in outlines}
    for i, attb in enumerate(attbs):
        # if we arent an outline and have right stroke-weight
        if i not in idxs and 'stroke-width' in attb and int(float(attb['stroke-width'])) == GRADIENT_WEIGHT:
            # check to see if middle point is in an outline
            r, c = curves[i][int(len(curves[i]) / 2)]
            r, c = int(r), int(c)
            for o in outlines:
                if o['mask'][r, c]:
                    normalized_path = normalize_path(attb['d'])
                    svg_path = parse_path(normalized_path)
                    # crop to boundary
                    t1, t2 = get_intersection_ts(svg_path, o['svg_border'])
                    if t1 is not None or t2 is not None:
                        if t1 is None: t1 = 0.
                        if t2 is None: t2 = 1.
                        svg_path = svg_path.cropped(t1, t2)
                    
                    
                    o['gradients'].append(curves[i])
                    o['colors'].append(attb['stroke'])
                    o['paths'].append(normalized_path)
                    o['svg_paths'].append(svg_path)
                   

In [304]:
def my_interp(p1, p2, steps):
    xs = np.linspace(p1[0], p2[0], steps)
    ys = np.linspace(p1[1], p2[1], steps)
    return list(zip(xs, ys))
            
def interpolate_curves(c1, c2, n=10):
    if len(c1) > len(c2):
        l1 = np.asarray(c1)
        l2 = np.asarray(c2)
    else:
        l1 = np.asarray(c2)
        l2 = np.asarray(c1)
    
    cs = []
    for i, p1 in enumerate(l1):
        spot = i / len(l1)
        j = int(len(l2) * spot)
        p2 = l2[j]
        cs.append(my_interp(p1, p2, n))
        
    curves = []
    for i in range(n):
        c = []
        for j in range(len(l1)):
            c.append((cs[j][i]))
        curves.append(c)
        
    return curves
        
        
def generate_full_gradient(gradients, n=50):
    full = []
    for i in range(len(gradients) - 1):
        full += interpolate_curves(gradients[i], gradients[i+1])
    return full

In [301]:
get_intersection_ts(a2, outlines[0]['svg_border'])

[((0.9869447179743355, QuadraticBezier(start=(180.103+27.875j), control=(180.371+26.571j), end=(180.616+25.469j)), 0.04626905918121338), (0.7898808630465532, QuadraticBezier(start=(188.727+29.155j), control=(184.325+28.484j), end=(179.962+27.726j)), 0.9810065031051636))]
0.9810065031051636


(None, 0.9810065031051636)

In [275]:
o1.d()

'M 56.471,70.353 L 57.278,70.721 L 59.07,71.316 Q 66.267,73.928 68.484,74.532 Q 70.701,75.136 74.968,75.884 Q 79.235,76.631 80.6,76.78 Q 81.965,76.929 84.888,77.135 Q 87.812,77.341 90.69,77.055 Q 93.568,76.768 95.727,76.268 Q 97.886,75.767 101.773,74.277 Q 105.66,72.787 109.037,70.999 Q 112.414,69.211 113.748,68.321 Q 115.083,67.43 116.42,66.394 Q 117.757,65.357 124.276,59.112 Q 130.794,52.867 134.143,49.301 Q 137.493,45.734 139.3,43.372 Q 141.108,41.009 142.171,39.355 Q 143.234,37.702 143.264,37.637 Q 143.294,37.572 144.357,35.719 Q 145.42,33.867 147.385,30.613 Q 149.35,27.36 149.614,26.88 Q 149.879,26.401 150.609,25.225 L 151.283,23.954'

In [305]:
curves = get_curves(paths)

outlines = get_outlines(attributes, curves)
add_gradients(attributes, curves, outlines)

[((0.9999729936728557, QuadraticBezier(start=(115.772+428.774j), control=(116.432+429.68j), end=(120.565+434.979j)), 0.9993698298931122), (0.25907975266242844, QuadraticBezier(start=(109.177+435.46j), control=(116.854+435.016j), end=(121.065+434.965j)), 0.9414252936840057))]
0.9414252936840057
[]
[((0.0, Line(start=(426.868+244.779j), end=(434.532+235.228j)), 0.0), (0.0, QuadraticBezier(start=(426.868+244.779j), control=(424.191+255.582j), end=(424.21+255.744j)), 0.0))]
0.0
[]
[]
[((0.9869447179743355, QuadraticBezier(start=(180.103+27.875j), control=(180.371+26.571j), end=(180.616+25.469j)), 0.04626905918121338), (0.7898808630465532, QuadraticBezier(start=(188.727+29.155j), control=(184.325+28.484j), end=(179.962+27.726j)), 0.9810065031051636))]
0.9810065031051636
[((0.00026834040776834784, QuadraticBezier(start=(35.264+120.798j), control=(35.769+121.261j), end=(36.312+121.544j)), 0.05082470178604126), (0.9588887203577885, QuadraticBezier(start=(36.478+116.208j), control=(34.743+122.7

In [306]:
outlines[0]

{'index': 0,
 'bounds': [(174.947, 30.51599999999999),
  (176.23699999999997, 29.95599999999999),
  (178.16099999999994, 29.459000000000003),
  (187.375, 28.638000000000005),
  (198.84500000000003, 28.208),
  (210.57299999999998, 27.662000000000006),
  (221.88, 27.064999999999998),
  (231.32500000000005, 26.25999999999999),
  (245.42100000000005, 23.99000000000001),
  (267.58799999999997, 19.74600000000001),
  (288.019, 16.034999999999997),
  (298.913, 14.401999999999987),
  (308.96799999999996, 13.314999999999998),
  (320.095, 12.01600000000002),
  (328.25300000000004, 10.889999999999986),
  (334.721, 10.136000000000024),
  (345.26099999999997, 9.62700000000001),
  (356.937, 9.73599999999999),
  (369.971, 10.940999999999974),
  (384.96299999999997, 13.329000000000008),
  (396.361, 16.224999999999994),
  (404.63699999999994, 19.189999999999998),
  (411.61400000000003, 22.514999999999986),
  (417.538, 26.354000000000013),
  (421.952, 30.242999999999995),
  (425.447, 34.18600000000001),


In [185]:
o1, o2 = outlines[0]['svg_paths'][1], outlines[0]['svg_paths'][2]
t = .5

In [186]:
n = o2.normal(t)
normal_line = Line(o2.point(t), o2.point(t) + n*max(H, W))
normal_line

Line(start=(132.491690651951+98.33774838409065j), end=(-263.6696292871915-635.5642945235962j))

In [343]:
def get_intersection_distance(o1, o2, t=.5):
    """intersection distance from o1 to o2"""
    n = o1.normal(t)
    normal = Line(o1.point(t), o1.point(t) + n*max(H, W))
#     print(normal)
#     i = o2.intersect(normal.reversed())
    i = o2.intersect(normal, justonemode=True)
#     print(i)
    if i:
#         print(i)
        p1 = i[0][1].point(i[0][2])
        p2 = o1.point(t)
        a, b = np.asarray([float(p1.real), float(p1.imag)]), np.asarray([float(p2.real), float(p2.imag)])
#         print(a, b)
        return np.linalg.norm(a-b)
        
    return None

In [241]:
o2, o1

(Path(Line(start=(47.453+93.259j), end=(47.74+93.319j)),
      QuadraticBezier(start=(47.74+93.319j), control=(47.919+93.374j), end=(51.279+95.295j)),
      QuadraticBezier(start=(51.279+95.295j), control=(54.64+97.217j), end=(57.973+98.851j)),
      QuadraticBezier(start=(57.973+98.851j), control=(61.306+100.485j), end=(64.502+102.027j)),
      QuadraticBezier(start=(64.502+102.027j), control=(67.699+103.568j), end=(72.17+105.176j)),
      QuadraticBezier(start=(72.17+105.176j), control=(76.642+106.783j), end=(82.875+107.752j)),
      QuadraticBezier(start=(82.875+107.752j), control=(89.107+108.722j), end=(93.423+108.775j)),
      QuadraticBezier(start=(93.423+108.775j), control=(97.738+108.828j), end=(101.39+108.264j)),
      QuadraticBezier(start=(101.39+108.264j), control=(105.041+107.7j), end=(107.773+107.1j)),
      QuadraticBezier(start=(107.773+107.1j), control=(110.505+106.501j), end=(112.603+106.016j)),
      QuadraticBezier(start=(112.603+106.016j), control=(114.701+105.532j

In [250]:
get_intersection_distance(o2, o1)

35.58432164141097

In [194]:
a = np.array((1,1,1))
b = np.array((2,2,2))
a

array([1, 1, 1])

In [265]:
o1.intersect(normal_line.reversed(), justonemode=True)

((0.5250755131660362,
  QuadraticBezier(start=(113.748+68.321j), control=(115.083+67.43j), end=(116.42+66.394j)),
  0.689028226236648),
 (0.957332947672169,
  Line(start=(-263.6696292871915-635.5642945235962j), end=(132.491690651951+98.33774838409065j)),
  0.957332947672169))

In [217]:
c = o1.intersect(normal_line.reversed())[0][0][1].point()
c

(115.0835+67.39375000000001j)

In [219]:
c.imag

67.39375000000001

In [188]:
outlines[0]['svg_border'].intersect(normal_line.reversed())

[((0.8620742281978655,
   QuadraticBezier(start=(85.242+10.148j), control=(80.219+10.094j), end=(77.86+10.394j)),
   0.03588817366311877),
  (0.8798297821610745,
   Line(start=(-263.6696292871915-635.5642945235962j), end=(132.491690651951+98.33774838409065j)),
   0.8798297821610745))]

In [22]:
def align_text_to_path(text, d, offset=0, method='align', length_adjust='spacing', spacing='exact',
                      font_family='Ariel', font_size='12px'):
    i = str(uuid.uuid4())
    p = etree.fromstring(f'<path id="{i}" fill="none" d="{d}" />')
    t = etree.fromstring(f'<text font-family="{font_family}" font-size="{font_size}">\
<textPath href="#{i}" startOffset="{offset}" method="{method}" \
lengthAdjust="{length_adjust}" spacing="{spacing}">{text}</textPath></text>')
    return t, p

In [273]:
b.cropped(o1, o2)

TypeError: '<=' not supported between instances of 'int' and 'Path'

In [350]:
r = etree.Element(
    "svg", width=svg_attributes['width'], height=svg_attributes['height'], xmlns="http://www.w3.org/2000/svg"
)
r.append(etree.fromstring('<defs><style>\
@import url("https://fonts.googleapis.com/css?family=Roboto:400,400i,700,700i");\
@import url("https://fonts.googleapis.com/css2?family=Special+Elite");</style></defs>'))
    
a1, a2 = None, None
# for i, (d, c) in enumeratezip(outlines[0]['paths'], outlines[0]['svg_paths']):
b = outlines[0]['svg_border']
outline = outlines[0]
for j in range(1, len(outline['paths']), 1):
# for j in range(len(outline['paths'])-1, 1, -1):
    d2 = outline['paths'][j]
#     print(d2)
    c1, c2 = outline['svg_paths'][j-1], outline['svg_paths'][j]
    chars = 'the brown fox jumps over the lazy dog'
    offset = 0
    for i, char in enumerate(chars):
        t = offset / c2.length()
        if t >= 1.:
            break
        
        dist1 = get_intersection_distance(c2, c1, t)
        dist2 = get_intersection_distance(c2, b, t)
#         normal = Line(c2.point(t), c2.point(t) + n*max(H, W))
        
        if dist1 is None:
            dist = dist2
        elif dist2 is None:
            dist = dist1
        else:
            dist = min(dist1, dist2)
        
        font_size = get_ideal_px(char, dist)
#         font_size = min(font_size, 28)
        print(j, char, dist1, dist2)
#         font_size = 12
        
        w = get_text_width(char, font_size)
        p, t = align_text_to_path(char, d2,
                                 font_family='Special Elite', font_size=f'{font_size}px', offset=offset)
        offset += w
        r.append(p)
        r.append(t)

1 t None 66.17818186406339
1 h None 68.43658311096897
1 e None 48.54548185174093
2 t None 8.983815479782299
2 h 25.639956166127984 96.54495168168559
2 e 29.58462435611985 98.29202204375106
2   32.0015519384154 97.7342624308574
2 b 32.86195943288098 99.11846618305918
2 r 35.162155765413665 100.18114397466728
2 o 36.64420642880894 99.26082739638159
2 w 33.96516278637827 108.74681315142567
2 n None 20.760267365961816
3 t 24.73912959792332 117.63017136819892
4 t None 232.37593455576018
4 h None 181.83032939997415
4 e None 104.79858021217353
4   None 47.86107377489235
5 t None 294.10432753358486
5 h 30.732773737024484 168.75350516378933
5 e 25.805957731164256 173.079468098174
5   21.866115463510916 181.7861785048434
5 b 20.60669651408802 192.26361388935823
5 r 19.88732086242881 204.47541416990143
5 o 19.884404143191887 204.66351165559612
5 w 19.918099845775625 199.45042002994225
5 n 19.031732239048374 180.36003666302867
5   19.022305444490744 148.5940833085099
5 f 19.343304401400815 134.374

In [23]:
from fontTools.ttLib import TTFont
font_fp = '/Users/erikstorrs/Downloads/Special_Elite/SpecialElite-Regular.ttf'
font = TTFont(font_fp)

In [351]:
display(SVG(etree.tostring(r).decode('utf-8')))

In [149]:
path_alt.point(.6)

(186.9559212327997+224.52561886861525j)

In [255]:
o1.length()

118.65553663849005

In [24]:
font

In [67]:
s = font.getGlyphSet()
t = cmap.getcmap(3,1).cmap

In [69]:
g = s[t[ord('A')]]
g

In [104]:
from fontTools.ttLib.tables._c_m_a_p import CmapSubtable

font = TTFont(font_fp)
def get_text_width(text, pointSize):
    cmap = font['cmap']
    t = cmap.getcmap(3,1).cmap
    s = font.getGlyphSet()
    units_per_em = font['head'].unitsPerEm
    total = 0
    for c in text:
        if ord(c) in t and t[ord(c)] in s:
            total += s[t[ord(c)]].width
        else:
            total += s['.notdef'].width
    total = total*float(pointSize)/units_per_em;
    return total

def get_text_height(text, pointSize):
    pillow_font = ImageFont.truetype(font_fp, pointSize)
    return pillow_font.getsize(text)[1]

def get_ideal_px(text, height):
    prev = 1
    for i in range(200):
        h = get_text_height(text, i)
        if h < height:
            prev = i
        elif h >= height:
            return prev
    return 200
    

# print ('Text: "%s"' % text)
# print ('Width in points: %f' % width)
# print ('Width in pixels: %f' % (width/72) * 96)
# print ('Width in inches: %f' % (width/72))
# print ('Width in cm: %f' % (width*2.54/72))
# print ('Width in WP Units: %f' % (width*1200/72))

In [105]:
get_ideal_px('h', 20)

25

In [80]:
get_text_width('e', 24, font)

13.0546875

In [318]:
r = etree.Element(
    "svg", width=svg_attributes['width'], height=svg_attributes['height'], xmlns="http://www.w3.org/2000/svg"
)
r.append(etree.fromstring('<defs><style>\
@import url("https://fonts.googleapis.com/css?family=Roboto:400,400i,700,700i");\
@import url("https://fonts.googleapis.com/css2?family=Special+Elite");</style></defs>'))
    
outline = outlines[0]
for d in outline['paths']:
    s = 'auto'
    m = 'stretch'
#     p, t = align_text_to_path('the brown fox jumps over the lazy dog', d, spacing=s, method=m,
#                              font_family='Special Elite')
    words = 'the brown fox jumps over the lazy dog'
    offset = 0
    for i, word in enumerate(words):
#         word = word + ' '
        font_size = 6 + i
        w = get_text_width(word, font_size)
        p, t = align_text_to_path(word, d,
                                 font_family='Special Elite', font_size=f'{font_size}px', offset=offset)
        offset += w
        r.append(p)
        r.append(t)

In [85]:
from PIL import Image, ImageDraw, ImageFont
f = ImageFont.truetype(font_fp, 12)
f




In [87]:
f.getsize('asdfjsdlk')

(58, 12)

In [319]:
display(SVG(etree.tostring(r).decode('utf-8')))

In [316]:
display(SVG(etree.tostring(r).decode('utf-8')))

In [ ]:
import lxml

In [ ]:
from lxml import etree
my_root_element = etree.Element(
    "svg", width=svg_attributes['width'], height=svg_attributes['height'], xmlns="http://www.w3.org/2000/svg"
)

In [ ]:
height = 100
my_rectangle = etree.fromstring(
    f'<rect class="some-css-class" x="13" y="14" width="500" height="{height}" />'
)

In [ ]:
my_root_element.append(my_rectangle)

In [ ]:
my_root_element

In [ ]:
etree.tostring(my_root_element)

In [ ]:
f = open('test.svg', 'w')
f.write(etree.tostring(my_root_element).decode('utf-8'))
f.close()

In [ ]:
etree.tostring(my_root_element).decode('utf-8')

In [ ]:
p = '<path id="path1" fill="none" stroke="red" \
d="M10,90 Q90,90 90,45 Q90,10 50,10 Q10,10 10,40 Q10,70 45,70 Q70,70 75,50" />'
t = '<text>\
<textPath href="#path1" startOffset="0">Quick brown fox jumps over the lazy dog.</textPath>\
</text>'

my_root_element.append(etree.fromstring(p))
my_root_element.append(etree.fromstring(t))

In [ ]:
from IPython.core.display import HTML, SVG, display

In [ ]:
display(SVG(etree.tostring(my_root_element).decode('utf-8')))

In [ ]:
def add_text_path(string, path):
    

In [ ]:
dwg = svgwrite.Drawing('test.svg', profile='full', size=(svg_attributes['width'] ,svg_attributes['height']))

dwg.add(dwg.rect((0, 0), (10, 10), fill='blue'))

In [ ]:
dwg.save()

In [ ]:
dwg.tostring()

In [ ]:
dwg

In [ ]:
w

In [ ]:
# w = outlines[0]['paths'][0]
w = dwg.path(d="M150 150 L2000 2000 L150 2000 Z", stroke="green")
text = dwg.add(svgwrite.text.Text("", style = 'font-size:40px'))
text.add(svgwrite.text.TextPath(path=w, text="blab blab bla bal", startOffset=None,
                                method='align', spacing='exact'))


In [ ]:
text = dwg.add(svgwrite.text.Text("blahblakshdfl kjasd;lfkjasd;fl kjsad;lfjk"))
text.add(svgwrite.text.TextPath(path=w, text="blab blab bla bal", startOffset=None,
                                method='align', spacing='exact'))

In [ ]:
dwg.save()

In [ ]:
dwg

In [ ]:
text = dwg.add(svgwrite.text.Text("asdf adsf as dfas fasd fas d", style = "font-size:40px; font-family:Arial"))

#### calculate gradient

In [ ]:
OUTLINE_WEIGHT = 2
GRADIENT_WEIGHT = 1

In [ ]:
fp = '../data/test2.svg'
paths, attributes, svg_attributes = svg2paths2(fp)

In [ ]:
svg_attributes

In [ ]:
paths

In [ ]:
q = paths[0][0]
q

In [ ]:
len(paths), len(attributes)

In [ ]:
MIN_X, MIN_Y, W, H = [int(float(x)) for x in svg_attributes['viewBox'].split(' ')]
MIN_X, MIN_Y, W, H

In [ ]:
def get_path_xy(p):
    s = re.sub(r'\(|\)|j', r'', str(p.start))
    x, y = s.split('+')
    x, y = float(x) - MIN_X, float(y) - MIN_Y
    return y, x # return in row, col format

def get_curve(paths):
    return [get_path_xy(p) for p in paths]

def get_curves(svg_paths):
    return [get_curve(ps) for ps in svg_paths]

In [ ]:
def get_outlines(attbs, curves):
    outlines = []
    for i, attb in enumerate(attbs):
        o = {}
        if 'stroke-width' in attb and int(float(attb['stroke-width'])) == OUTLINE_WEIGHT:
            o = {'index': i, 'bounds':curves[i], 'metadata': attb}
#             print(curves[i]['bounds'])

            o['polygon'] = draw.polygon([r for r, c in o['bounds']],
                                        [c for r, c in o['bounds']])
            o['mask'] = np.full((H, W), False)
            o['mask'][o['polygon'][0], o['polygon'][1]] = True
            o['gradients'] = []
        if o:
            outlines.append(o)
    return outlines

def add_gradients(attbs, curves, outlines):
    idxs = {o['index'] for o in outlines}
    for i, attb in enumerate(attbs):
        # if we arent an outline and have right stroke-weight
        if i not in idxs and 'stroke-width' in attb and int(float(attb['stroke-width'])) == GRADIENT_WEIGHT:
            # check to see if middle point is in an outline
            r, c = curves[i][int(len(curves[i]) / 2)]
            r, c = int(r), int(c)
            for o in outlines:
                if o['mask'][r, c]:
                    o['gradients'].append(curves[i])
                    o['color'] = attb['stroke']

In [ ]:
def my_interp(p1, p2, steps):
    xs = np.linspace(p1[0], p2[0], steps)
    ys = np.linspace(p1[1], p2[1], steps)
    return list(zip(xs, ys))
            
def interpolate_curves(c1, c2, n=10):
    if len(c1) > len(c2):
        l1 = np.asarray(c1)
        l2 = np.asarray(c2)
    else:
        l1 = np.asarray(c2)
        l2 = np.asarray(c1)
    
    cs = []
    for i, p1 in enumerate(l1):
        spot = i / len(l1)
        j = int(len(l2) * spot)
        p2 = l2[j]
        cs.append(my_interp(p1, p2, n))
        
    curves = []
    for i in range(n):
        c = []
        for j in range(len(l1)):
            c.append((cs[j][i]))
        curves.append(c)
        
    return curves
        
        
def generate_full_gradient(gradients, n=50):
    full = []
    for i in range(len(gradients) - 1):
        full += interpolate_curves(gradients[i], gradients[i+1])
    return full

In [ ]:
grads = generate_full_gradient(outlines[0]['gradients'])

In [ ]:
for cv in grads:
    plt.scatter([r for r, c in cv], [c for r, c in cv])

In [ ]:
for cv in outlines[0]['gradients']:
    plt.scatter([r for r, c in cv], [c for r, c in cv])

In [ ]:
new = interpolate_curves(c1, c2)
len(new)

In [ ]:
from scipy.interpolate import UnivariateSpline
spl = UnivariateSpline([1,2,3,4,5], [1,2,3.5,4.5,5])
spl.get_coeffs()

In [ ]:
spl = UnivariateSpline([x for x, y in new[0]], [y for x, y in new[0]])
spl.get_coeffs()

In [ ]:
from svgpathtools import polynomial2bezier

In [ ]:
polynomial2bezier(spl.get_coeffs())

In [ ]:
polynomial2bezier(spl.get_coeffs())

In [ ]:
for cv in [c1, c2]:
    plt.scatter([r for r, c in cv], [c for r, c in cv])

In [ ]:
for cv in new:
    plt.scatter([r for r, c in cv], [c for r, c in cv])

In [ ]:
c1 = outlines[0]['gradients'][0]
c2 = outlines[0]['gradients'][1]



In [ ]:
for cv in outlines[0]['gradients']:
    plt.scatter([r for r, c in cv], [c for r, c in cv])

In [ ]:
from skimage import draw

In [ ]:
curves = get_curves(paths)

In [ ]:
outlines = get_outlines(attributes, curves)
add_gradients(attributes, curves, outlines)

In [ ]:
for cv in outlines[0]['gradients']:
    plt.scatter([r for r, c in cv], [c for r, c in cv])

In [ ]:
# clip to polygon
# extend to polygon

# iterate until you have a bunch of lines

# start at curve 0
# - find smallest curve that can fit a word
# - move from that curve to find next curve that can fit a word at closest point
# 


In [ ]:
for o in outlines:
    plt.imshow(o['mask'])
    plt.show()

In [ ]:
get_path_xy(q)

In [ ]:
outlines[0]

In [ ]:
attributes

In [ ]:
min_x, min_y, w, h = [float(x) for x in svg_attributes['viewBox'].split(' ')]
min_x, min_y, w, h

In [ ]:
p = attributes[0]
p

#### sandbox

In [ ]:
fp = '../data/test2.svg'
paths, attributes, svg_attributes = svg2paths2(fp)

In [ ]:
paths

In [ ]:
attributes

In [ ]:
svg_attributes

In [ ]:
~ls ./

In [ ]:
dwg.save('testtest.svg')

In [ ]:
svgwrite.ge

In [ ]:
from IPython.core.display import HTML, SVG, display

In [ ]:
dwg = svgwrite.Drawing()

In [ ]:
square = dwg.rect((0, 0), (10, 10), fill='blue')

In [ ]:
dwg

In [ ]:
import svgwrite
dwg = svgwrite.Drawing('test.svg', profile='full', size=(20 ,20))
dwg.add(dwg.line((0, 0), (10, 0), stroke=svgwrite.rgb(10, 10, 16, '%')))
dwg.add(dwg.text('Test', insert=(0, 0.2)))
dwg.save()

In [ ]:
dwg.tostring()

In [ ]:
display(SVG(dwg.tostring()))

In [ ]:
dwg

In [ ]:
svg_attributes

In [ ]:
attributes

In [ ]:
len(attributes)

In [ ]:
attributes[0].keys()

In [ ]:
dwg = svgwrite.Drawing('test.svg', profile='full', size=(svg_attributes['width'] ,svg_attributes['height']))

In [ ]:
dwg.tostring()

In [ ]:
dwg.add(dwg.textPath(attributes[0]['d'],
                               'hello world skdjfh sdkjfh sdkjfh '))

In [ ]:
dwg.textPath

In [ ]:
text = dwg.add(svgwrite.text.Text(""))
text.add(svgwrite.text.TextPath(path=attributes[0]['d'], text="blab blab bla bal",
                                startOffset=None, method='align', spacing='exact',
                               style = "font-size:40px; font-family:Arial"))

In [ ]:
text = dwg.add(svgwrite.text.Text("asdf adsf as dfas fasd fas d", style = "font-size:40px; font-family:Arial"))

In [ ]:
dwg.tostring()

In [ ]:
display(SVG(dwg.tostring()))

In [ ]:
dwg.save()